$$(*) \, ||b-Ax||_2 \to \min!$$
$$(**) \, ||b-APy||_2 \to \min!, x=Py$$
1) Wygenerować rozrzedzoną $A \in \R^{M \times N}$ (scipy.sparse) i $b \in \R^N$

2) Rozwiązać (*) metodą CGNR/LSQR

3) Rozwiązać (**) metodą j.w. dla $P$ - niepełny rozkład Cholesky'ego $A^TA$

4) Porównać szybkości zbieżności

In [101]:
import numpy as np
import scipy as sp

A = sp.sparse.random(20, 10, density=0.2, format='csr', dtype='float64', random_state=None)
b = np.random.rand(20)
print(f"A={A.todense()}")
print(f"b={b}")

A=[[0.         0.         0.         0.         0.36500078 0.
  0.38417469 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.44927427 0.         0.74661636]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.47780378]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.8111344  0.        ]
 [0.         0.         0.         0.         0.         0.03822562
  0.08267713 0.         0.         0.        ]
 [0.         0.         0.         0.         0.15360281 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.11894059
  0.         0.         0.48266367 0.        ]
 [0.         0.8095972  0.         0.         0.         0.
  0.         0.         0.89109649 0.        ]
 [0.         0.76401159 0.07062233 0.         0.41548549 0.
  0.         0.         0.         0.85861136]
 [0.         0.0638

In [102]:
def matvec(x):
    return A.T @ (A @ x)

x_cg, _ = sp.sparse.linalg.cg(sp.sparse.linalg.LinearOperator((10, 10), matvec=matvec), A.T @ b)
print(f"x={x_cg}")

x=[ 0.92030916  0.35329851  0.08752297  0.26494702  0.66432892  3.64605152
  0.14535382  0.44041078  0.1272463  -0.18534614]


In [103]:
x_lsqr = sp.sparse.linalg.lsqr(A, b)[0]
print(f"x={x_lsqr}")

x=[ 0.92030916  0.35329851  0.08752296  0.26494702  0.66432891  3.64605152
  0.14535382  0.44041078  0.12724629 -0.18534614]


In [104]:
x_real = np.linalg.lstsq(A.todense(), b, rcond=None)[0]
print(f"x={x_real}")

x=[ 0.92030916  0.35329851  0.08752296  0.26494702  0.66432891  3.64605152
  0.14535382  0.44041078  0.12724629 -0.18534614]


In [105]:
print(f"norm={np.linalg.norm(x_real - x_lsqr)}, {np.linalg.norm(x_real - x_cg)}, {np.linalg.norm(x_lsqr - x_cg)}")

norm=6.1639547730740885e-09, 7.043115114768326e-09, 1.3207069815767218e-08


3a) Wyznaczyć $P: P^TA^TAPx = P^TA^Tb$

3b) $C_L = LinearOperator(...)$, $matvec: P^T(A^T(A(Px)))$

3c) $x=cg(C_L, ...)[0]$

Niestety nie istnieje gotowa funkcja do niepełnego Cholesky'ego, ale istnieje do niepełnego LU!

In [106]:
B= A.T @ A
incompleteLU = sp.sparse.linalg.spilu(B)
L, U = incompleteLU.L, incompleteLU.U

Szukamy $\hat L$, tż. $B \approx LU = \hat L \hat L^T$

$L \cdot U = L \cdot diag(u_{1,1}, ..., u_{N,N}) \cdot L^T = (L \cdot D) \cdot (D \cdot L^T), \text{ gdzie } D = diag(\sqrt{u_{1,1}}, ..., \sqrt{u_{N,N}})$

In [109]:
D = sp.sparse.diags(np.sqrt(U.diagonal()))
L = L @ D

def matvec_incomplete(x):
    return sp.sparse.linalg.spsolve(L, A.T @ (A @ sp.sparse.linalg.spsolve(L.T, x)))

x_cg_incomplete, _ = sp.sparse.linalg.cg(sp.sparse.linalg.LinearOperator((10, 10), matvec=matvec_incomplete), sp.sparse.linalg.spsolve(L, A.T @ b))
print(x_cg_incomplete)
# callback

[ 4.29494712e+00  2.60536254e+00  1.25718374e-03  4.21755658e-01
  2.41437705e+00  3.85075227e+00  1.07661409e-01  4.09298287e-01
  3.66555345e-01 -1.29660917e-01]
